In [14]:
treatmentMetadataFile <- "/home/bioinf/bhklab/jermiah/projects/annotationScripts/rawdata/ctrp/v20.meta.per_compound.txt"
treatmentMetadata <- data.table::fread(treatmentMetadataFile, sep = "\t", check.names=TRUE)


In [15]:
treatmentMetadata[1:5]

master_cpd_id,cpd_name,broad_cpd_id,top_test_conc_umol,cpd_status,inclusion_rationale,gene_symbol_of_protein_target,target_or_activity_of_compound,source_name,source_catalog_id,cpd_smiles
<int>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1788,CIL55,BRD-K46556387,10,probe,pilot-set,,screening hit,Columbia University,,CN(C)CCNC(=O)c1cc2CSc3cc(Cl)ccc3-c2s1
3588,BRD4132,BRD-K86574132,160,probe,chromatin;pilot-set,,screening hit,ChemDiv Inc.,4998-1380,CC(C)N1C(=O)S\C(=C\c2ccc(Sc3nc4ccccc4[nH]3)o2)C1=O
12877,BRD6340,BRD-K35716340,33,probe,chromatin;pilot-set,,screening hit,ChemDiv Inc.,1988-0090,C(Cn1c2ccccc2c2ccccc12)c1nc2ccccc2[nH]1
17712,ML006,BRD-K89692698,530,probe,pilot-set,S1PR3,agonist of sphingosine 1-phosphate receptor 3,Enamine Ltd.,Z1037336336,C1CN(CCO1)c1nnc(-c2ccccc2)c(n1)-c1ccccc1
18311,Bax channel blocker,BRD-A18763547,33,probe,pilot-set,BAX,inhibitor of BAX-mediated mitochondrial cytochrome c release,Maybridge,RJC01737,OC(CN1CCNCC1)Cn1c2ccc(Br)cc2c2cc(Br)ccc12


In [16]:
# clean the treatment names and create a column called `CTRP.cleanedTreatmentNames`
treatmentMetadata <- treatmentMetadata[, "CTRP.cleanedTreatmentNames" := cleanCharacterStrings(cpd_name)]
treatmentMetadata[1:5]


master_cpd_id,cpd_name,broad_cpd_id,top_test_conc_umol,cpd_status,inclusion_rationale,gene_symbol_of_protein_target,target_or_activity_of_compound,source_name,source_catalog_id,cpd_smiles,CTRP.cleanedTreatmentNames
<int>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1788,CIL55,BRD-K46556387,10,probe,pilot-set,,screening hit,Columbia University,,CN(C)CCNC(=O)c1cc2CSc3cc(Cl)ccc3-c2s1,CIL55
3588,BRD4132,BRD-K86574132,160,probe,chromatin;pilot-set,,screening hit,ChemDiv Inc.,4998-1380,CC(C)N1C(=O)S\C(=C\c2ccc(Sc3nc4ccccc4[nH]3)o2)C1=O,BRD4132
12877,BRD6340,BRD-K35716340,33,probe,chromatin;pilot-set,,screening hit,ChemDiv Inc.,1988-0090,C(Cn1c2ccccc2c2ccccc12)c1nc2ccccc2[nH]1,BRD6340
17712,ML006,BRD-K89692698,530,probe,pilot-set,S1PR3,agonist of sphingosine 1-phosphate receptor 3,Enamine Ltd.,Z1037336336,C1CN(CCO1)c1nnc(-c2ccccc2)c(n1)-c1ccccc1,ML006
18311,Bax channel blocker,BRD-A18763547,33,probe,pilot-set,BAX,inhibitor of BAX-mediated mitochondrial cytochrome c release,Maybridge,RJC01737,OC(CN1CCNCC1)Cn1c2ccc(Br)cc2c2cc(Br)ccc12,BAXCHANNELBLOCKER


In [53]:
THREADS <- 10
compound_nameToCIDS <- 
    AnnotationGx::getPubChemCompound(
        treatmentMetadata[1:100, cpd_name], 
        from='name', 
        to='cids', 
        batch = FALSE,
        verbose = FALSE,
        BPPARAM = BiocParallel::MulticoreParam(workers = THREADS, progressbar = TRUE, stop.on.error = FALSE)
    )


  |======================================================================| 100%



In [54]:
compound_nameToCIDS <- compound_nameToCIDS[!duplicated(name),] 
data.table::setnames(compound_nameToCIDS, "name", "cpd_name")
str(compound_nameToCIDS)


Classes ‘data.table’ and 'data.frame':	99 obs. of  2 variables:
 $ cpd_name: chr  "CIL55" "BRD4132" "BRD6340" "ML006" ...
 $ cids    : int  6623618 7326481 1641662 2842253 2729026 613000 444795 64971 4788 5426 ...
 - attr(*, ".internal.selfref")=<externalptr> 
 - attr(*, "failed")=List of 1
  ..$ :List of 2
  .. ..$ query  : chr "compound 1B"
  .. ..$ failure:List of 1
  .. .. ..$ Fault:List of 3
  .. .. .. ..$ Code   : chr "PUGREST.NotFound"
  .. .. .. ..$ Message: chr "No CID found"
  .. .. .. ..$ Details: chr "No CID found that matches the given name"


In [57]:
# there was 99 successful and 1 failure :
data.table::rbindlist(attributes(compound_nameToCIDS)$failed, fill = TRUE)  # get all the failed queries

query,failure
<chr>,<list>
compound 1B,PUGREST.....


In [58]:
# The raw metadata has a column for SMILES which we can also use
## ----------------- getPubChemCompound using SMILES from failed ----------------- ##

# first get the row(s) from the metadata file that corresponds to the failed query
failed_dt <- merge(
    treatmentMetadata, 
    data.table::rbindlist(attributes(compound_nameToCIDS)$failed, fill = TRUE),  # get all the failed queries
    by.x = "cpd_name", 
    by.y = "query")
failed_dt |> str()

Classes ‘data.table’ and 'data.frame':	1 obs. of  13 variables:
 $ cpd_name                      : chr "compound 1B"
 $ master_cpd_id                 : int 62111
 $ broad_cpd_id                  : chr "BRD-A42556028"
 $ top_test_conc_umol            : num 66
 $ cpd_status                    : chr "probe"
 $ inclusion_rationale           : chr "chromatin;outreach"
 $ gene_symbol_of_protein_target : chr ""
 $ target_or_activity_of_compound: chr "screening hit"
 $ source_name                   : chr "Enamine Ltd."
 $ source_catalog_id             : chr "T0503-7331"
 $ cpd_smiles                    : chr "CCN(CC)c1ccc(cc1[N+]([O-])=O)C1=NNC(=O)CC1C"
 $ CTRP.cleanedTreatmentNames    : chr "COMPOUND1B"
 $ failure                       :List of 1
  ..$ :List of 3
  .. ..$ Code   : chr "PUGREST.NotFound"
  .. ..$ Message: chr "No CID found"
  .. ..$ Details: chr "No CID found that matches the given name"
 - attr(*, ".internal.selfref")=<externalptr> 
 - attr(*, "sorted")= chr "cpd_name"


In [60]:
smilesToCIDS <- 
    AnnotationGx::getPubChemCompound(
        failed_dt$cpd_smiles, 
        from='smiles', 
        to='cids', 
        batch = FALSE,
        verbose = FALSE,
        BPPARAM = BiocParallel::MulticoreParam(workers = THREADS, progressbar = TRUE, stop.on.error = FALSE)
    )
smilesToCIDS <- smilesToCIDS[!duplicated(smiles), ]

# Merge the successful smiles queries with the original metadata to get the cpd_name 
smiles_successful_dt <- merge(
    treatmentMetadata, 
    smilesToCIDS, 
    by.x = "cpd_smiles", 
    by.y = "smiles")[, c("cpd_name", "cids")]
smiles_successful_dt 

  |======================================================================| 100%



cpd_name,cids
<chr>,<int>
compound 1B,2874250


In [61]:
# Combine the successful smiles queries with the successful name queries
successful_CTRP.master_cpd_ids <- 
    data.table::rbindlist(
        list(
            compound_nameToCIDS, 
            smiles_successful_dt), 
        fill = TRUE
    )
# view the 
successful_CTRP.master_cpd_ids |> str()
successful_CTRP.master_cpd_ids[1:5]

Classes ‘data.table’ and 'data.frame':	100 obs. of  2 variables:
 $ cpd_name: chr  "CIL55" "BRD4132" "BRD6340" "ML006" ...
 $ cids    : int  6623618 7326481 1641662 2842253 2729026 613000 444795 64971 4788 5426 ...
 - attr(*, ".internal.selfref")=<externalptr> 


cpd_name,cids
<chr>,<int>
CIL55,6623618
BRD4132,7326481
BRD6340,1641662
ML006,2842253
Bax channel blocker,2729026


# Use CID to get other annotations

We will only use the first 5 cids here

In [66]:
source("/home/bioinf/bhklab/jermiah/projects/annotationScripts/scripts/getPubChem/newfunctions.R")


Attaching package: ‘data.table’


The following object is masked _by_ ‘.GlobalEnv’:

    .N




In [78]:
annotations <- c('ChEMBL ID', 'NSC Number', 'Drug Induced Liver Injury', 'CAS', 'ATC Code')

annotationType = annotations[1]

result <- 
    lapply(
        annotations,
        function(annotationType){
            data.table::rbindlist(
                BiocParallel::bptry(
                    BiocParallel::bplapply(
                            successful_CTRP.master_cpd_ids[1:5,cids], 
                            function(CID) {
                                data.table::as.data.table(getPubChemCHEMBL(CID, type = annotationType))
                            },
                            BPPARAM = BiocParallel::MulticoreParam(workers = THREADS, progressbar = TRUE, stop.on.error = FALSE)
                    )
                ),
                fill=TRUE
            )
        }
    )


  |========================================================              |  80%

R_zmq_msg_send errno: 4 strerror: Interrupted system call




  |======================================================================| 100%

  |                                                                      |   0%

R_zmq_msg_send errno: 4 strerror: Interrupted system call




  |==============                                                        |  20%

...

  |==========================================                            |  60%

R_zmq_msg_send errno: 4 strerror: Interrupted system call




  |========================================================              |  80%

R_zmq_msg_send errno: 4 strerror: Interrupted system call




  |======================================================================| 100%

  |======================================================================| 100%

  |======================================================================| 100%

  |======================================================================| 100%



In [86]:

# merge all the data together on the "cid" column 
data <- Reduce(function(x, y) merge(x, y, by = "cid", all = TRUE), result)
data <- merge(successful_CTRP.master_cpd_ids, data, by.x = "cids", by.y="cid", all = FALSE)
data

cids,cpd_name,ChEMBL ID,NSC Number,Drug Induced Liver Injury,CAS,ATC Code
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1641662,BRD6340,NA,NA,NA,N/A,NA
2729026,Bax channel blocker,CHEMBL1402326,NA,NA,DTXSID90369557,NA
2842253,ML006,CHEMBL1526042,NA,NA,N/A,NA
6623618,CIL55,CHEMBL492468,NA,NA,N/A,NA
7326481,BRD4132,CHEMBL1300397,NA,NA,N/A,NA


In [76]:
CIDS_to_Annotation <- rbindlist(result, fill = TRUE)
CIDS_to_Annotation

cid,ChEMBL ID
<int>,<chr>
6623618,CHEMBL492468
7326481,CHEMBL1300397
1641662,NA
2842253,CHEMBL1526042
2729026,CHEMBL1402326
